In [ ]:
from config import myconfig
from binance.client import Client
import pandas as pd
from tqdm import tqdm
import os
from datetime import datetime

# Get the absolute path of the script directory (Jupyter compatible)
SCRIPT_DIR = os.getcwd()

# Define the folder structure based on the selected Kline interval
DATA_FOLDER = os.path.join(SCRIPT_DIR, "data")

# Create required directories if they don't exist
os.makedirs(DATA_FOLDER, exist_ok=True)

# Generate timestamp for file versioning
timestamp = datetime.utcnow().strftime('%Y-%m-%d_%H-%M-%S')  # Format: YYYY-MM-DD_HH-MM-SS

# Define CSV file path with timestamp to avoid overwriting
csv_path = os.path.join(DATA_FOLDER, f"binance_trading_volume_{timestamp}.csv")

# Read API keys
API_KEY = myconfig.BINANCE_API_KEY
API_SECRET = myconfig.BINANCE_API_SECRET

# Initialize Binance client
client = Client(API_KEY, API_SECRET)

# Fetch all trading pairs and include detailed metadata
exchange_info = client.get_exchange_info()
symbols_info = []

for s in exchange_info['symbols']:
    if s['status'] == 'TRADING':
        # Extract price and lot size filters
        price_filter = next((f for f in s['filters'] if f['filterType'] == 'PRICE_FILTER'), {})
        lot_size_filter = next((f for f in s['filters'] if f['filterType'] == 'LOT_SIZE'), {})
        min_notional_filter = next((f for f in s['filters'] if f['filterType'] == 'MIN_NOTIONAL'), {})

        symbols_info.append({
            'symbol': s['symbol'],
            'baseAsset': s['baseAsset'],
            'quoteAsset': s['quoteAsset'],
            'minPrice': price_filter.get('minPrice', 'N/A'),
            'maxPrice': price_filter.get('maxPrice', 'N/A'),
            'tickSize': price_filter.get('tickSize', 'N/A'),
            'minQty': lot_size_filter.get('minQty', 'N/A'),
            'maxQty': lot_size_filter.get('maxQty', 'N/A'),
            'stepSize': lot_size_filter.get('stepSize', 'N/A'),
            'minNotional': min_notional_filter.get('minNotional', 'N/A'),
            'status': s['status'],
            'allowedOrderTypes': ", ".join(s['orderTypes'])
        })

# Fetch 24-hour trading volume for all active pairs with a progress bar
volume_data = []

for symbol_info in tqdm(symbols_info, desc="Fetching trading volume", unit="pair"):
    try:
        ticker = client.get_ticker(symbol=symbol_info['symbol'])
        volume_data.append({
            'timestamp': timestamp,
            'symbol': symbol_info['symbol'],
            'baseAsset': symbol_info['baseAsset'],
            'quoteAsset': symbol_info['quoteAsset'],
            'minPrice': symbol_info['minPrice'],
            'maxPrice': symbol_info['maxPrice'],
            'tickSize': symbol_info['tickSize'],
            'minQty': symbol_info['minQty'],
            'maxQty': symbol_info['maxQty'],
            'stepSize': symbol_info['stepSize'],
            'minNotional': symbol_info['minNotional'],
            'status': symbol_info['status'],
            'allowedOrderTypes': symbol_info['allowedOrderTypes'],
            'baseAssetVolume': float(ticker['volume']),  # Volume in base asset
            'quoteAssetVolume': float(ticker['quoteVolume'])  # Volume in quote asset
        })
    except Exception as e:
        print(f"Error fetching data for {symbol_info['symbol']}: {e}")

# Convert to DataFrame
df = pd.DataFrame(volume_data)

# Sort by highest quote volume
df = df.sort_values(by='quoteAssetVolume', ascending=False)

# Save DataFrame to a **new CSV file** each run
df.to_csv(csv_path, index=False)

print(f"Data successfully saved to {csv_path}")


In [14]:
import pandas as pd
import openai
import os
from config import myconfig

# Get script directory and define paths
SCRIPT_DIR = os.getcwd()
DATA_FOLDER = os.path.join(SCRIPT_DIR, "data")
os.makedirs(DATA_FOLDER, exist_ok=True)

csv_file_name = "binance_trading_volume_2025-02-11_15-01-43.csv"
csv_file_path = os.path.join(DATA_FOLDER, csv_file_name)

# Load OpenAI API key
OPENAI_API_KEY = myconfig.OPENAI_API_KEY
OPENAI_ORG = myconfig.OPENAI_ORG
OPENAI_PROJECT = myconfig.OPENAI_PROJECT
client = openai.OpenAI(api_key=OPENAI_API_KEY, organization=OPENAI_ORG, project=OPENAI_PROJECT)

# Check if the file is already uploaded
existing_files = client.files.list().data  # Retrieve uploaded files
file_id = None

for file in existing_files:
    if file.filename == csv_file_name:  # Correct filename check
        file_id = file.id
        print(f"File already uploaded. Using existing File ID: {file_id}")
        break

# Upload if file is not found
if not file_id:
    with open(csv_file_path, "rb") as file:
        uploaded_file = client.files.create(
            file=file,
            purpose="assistants"
        )
    file_id = uploaded_file.id
    print(f"File uploaded successfully! File ID: {file_id}")


pytrade_assistant = client.beta.assistants.create(
  name="Py Trade Bot",
  description="You are a trading assistant. Use your knowledge to analyze market data.",
  model="gpt-4o",
  tools=[{"type": "code_interpreter"}],
  tool_resources={
    "code_interpreter": {
      "file_ids": [file_id]
    }
  }
)

# Create a new thread
thread = client.beta.threads.create()

# Send the file for analysis (Now includes 'tools' inside 'attachments')
prompt = "Analyze the attached Binance trading volume data and provide insights."
my_thread_message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    attachments=[{
          "file_id": file_id,
          "tools": [{"type": "code_interpreter"}]
        }],
    content=prompt,
)

# Run the assistant on this thread
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=pytrade_assistant.id, timeout=1000
)

# Check if the run was successful
if run.status != "completed":
    raise Exception("Run failed:", run.status)

# Retrieve and print the response
messages_cursor = client.beta.threads.messages.list(thread_id=thread.id)
messages = [message for message in messages_cursor]

# Output text
res_txt = messages[0].content[0].text.value
print(res_txt)


File already uploaded. Using existing File ID: file-XUDhLicC7CRh51LJyVc6V8
Here are some insights from the Binance trading volume data:

### Summary Statistics

- **Base Asset Volume**
  - **Mean:** 24.044 billion units
  - **Median:** 788,283.5 units
  - **Maximum:** 20.926 trillion units
  
- **Quote Asset Volume**
  - **Mean:** 28.806 million units
  - **Median:** 914,718.8 units
  - **Maximum:** 3.272 billion units

### Top Trading Pairs by Volume

#### By Base Asset Volume:
1. **PEPEUSDT**: 20.926 trillion
2. **SHIBUSDT**: 2.006 trillion
3. **PEPETRY**: 1.723 trillion
4. **PEPEFDUSD**: 1.711 trillion
5. **BTTCUSDT**: 1.677 trillion
6. **BONKUSDT**: 1.554 trillion
7. **PEPEUSDC**: 892.055 billion
8. **FLOKIUSDT**: 293.965 billion
9. **BTTCTRY**: 222.690 billion
10. **LUNCUSDT**: 162.821 billion

#### By Quote Asset Volume:
1. **USDTTRY**: 3.273 billion
2. **BTCFDUSD**: 2.344 billion
3. **USDTARS**: 1.974 billion
4. **BNXTRY**: 1.634 billion
5. **BTCUSDT**: 1.461 billion
6. **USDTCO

In [10]:
import pandas as pd
import openai
import os
from config import myconfig

# Get script directory and define paths
SCRIPT_DIR = os.getcwd()
DATA_FOLDER = os.path.join(SCRIPT_DIR, "data")
os.makedirs(DATA_FOLDER, exist_ok=True)

csv_file_name = "binance_trading_volume_2025-02-11_15-01-43.csv"
csv_file_path = os.path.join(DATA_FOLDER, csv_file_name)

# Load OpenAI API key
OPENAI_API_KEY = myconfig.OPENAI_API_KEY
OPENAI_ORG = myconfig.OPENAI_ORG
OPENAI_PROJECT = myconfig.OPENAI_PROJECT

df = pd.read_csv(csv_file_path)

# Preview the data
print(df.head())

# Convert DataFrame to a structured format for analysis
summary_data = df.describe().to_string()

# Define the prompt for OpenAI
prompt = f"""
I have the following market data summary:
{summary_data}

Analyze the trends and provide insights regarding:
1. Overall market movement
2. Notable fluctuations in price or volume
3. Any patterns or anomalies you detect
4. Suggestions for potential trading strategies
"""

# Call OpenAI API (Updated for v1.0+)
client = openai.OpenAI(api_key=OPENAI_API_KEY, organization=OPENAI_ORG, project=OPENAI_PROJECT)

response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
        {"role": "system", "content": "You are a financial analyst."},
        {"role": "user", "content": prompt},
    ]
)

# Extract and print response
analysis = response.choices[0].message.content
print("\nMarket Analysis Report:\n")
print(analysis)


             timestamp    symbol baseAsset quoteAsset  minPrice   maxPrice  \
0  2025-02-11_15-01-43   USDTTRY      USDT        TRY      0.01     1000.0   
1  2025-02-11_15-01-43  BTCFDUSD       BTC      FDUSD      0.01  1000000.0   
2  2025-02-11_15-01-43   USDTARS      USDT        ARS      0.10   100000.0   
3  2025-02-11_15-01-43    BNXTRY       BNX        TRY      0.01    10000.0   
4  2025-02-11_15-01-43   BTCUSDT       BTC       USDT      0.01  1000000.0   

   tickSize   minQty      maxQty  stepSize  minNotional   status  \
0      0.01  1.00000  92141578.0   1.00000          NaN  TRADING   
1      0.01  0.00001      9000.0   0.00001          NaN  TRADING   
2      0.10  1.00000    922327.0   1.00000          NaN  TRADING   
3      0.01  0.10000   9222449.0   0.10000          NaN  TRADING   
4      0.01  0.00001      9000.0   0.00001          NaN  TRADING   

                                   allowedOrderTypes  baseAssetVolume  \
0  LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS, STOP_LO